In [99]:
from neo4j import GraphDatabase

In [111]:
class WalletDB(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password), encrypted=False)#REVISE LATER FOR ENCRYPTION!!!

    def close(self):
        self._driver.close()
        
    @staticmethod
    def _new_address(tx, address,i,change_addr):
        if change_addr: addr_type = "change"
        else: addr_type = "recipient"
        result = tx.run("CREATE a = (:address {address:$address, acc_index:$index, type:$kind, created:timestamp()}) "
                        "RETURN a ", address=address, index = i, kind = addr_type)
        return result.single()
    
    @staticmethod
    def _new_utxo(tx, address,tx_id,out_index):
        local_index = tx.run( "MATCH (u:utxo) RETURN COUNT (u) ").single()[0] 
        result = tx.run("MATCH (n:address {address : $address}) "
                        "CREATE p = (:utxo {address:$address, transaction_id:$tx_id, out_index:$out_index, local_index:$local_index, spent:false})-[:BELONGS]->(n) "
                        "RETURN p ", address=address, tx_id = tx_id, out_index = out_index, local_index=local_index)
        return result.single()
    
    
    @staticmethod
    def _new_tx(tx,tx_id,inputs, outputs):
        result = tx.run("CREATE (n:TxOut {id:$tx_id, inputs:$inputs , outputs:$outputs, confirmations:0, created:timestamp()}) "
                        "WITH n "
                        "UNWIND n.inputs AS ins "
                        "MATCH coins = (:utxo {local_index:ins}) "
                        "FOREACH ( coin IN nodes(coins) | CREATE (coin)<-[:SPENT]-(n) ) ",
                        tx_id=tx_id, inputs = inputs, outputs = outputs)
        return result.single()
    
    
    @staticmethod
    def _update_confirmations(tx,tx_id,n_confirmations):
        result = tx.run("MATCH (n:TxOut {id:$tx_id}) "
                        "SET n.confirmations=$n_confirmations "
                        "RETURN n ", tx_id=tx_id, n_confirmations = n_confirmations)
        return result.single()
    
    
    @staticmethod
    def _update_utxo(tx,tx_id):
        result = tx.run("MATCH p = (u)<-[:SPENT]-(:TxOut {id:$tx_id}) "
                        "SET u.spent=true "
                        "RETURN p ", tx_id=tx_id)
        return result.single()
    


    
    def new_address(self, address,i,change_addr):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_address, address,i,change_addr)
            print(result)
            
    def new_utxo(self, address,tx_id,out_index):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_utxo, address,tx_id,out_index)
            print(result)
            
    def new_tx(self, tx_id,utxo_local_index_list, outputs):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_tx, tx_id,utxo_local_index_list, outputs)
            print(result)
            
    def update_confirmations(self, tx_id,n_confirmations):
        with self._driver.session() as session:
            result = session.write_transaction(self._update_confirmations, tx_id,n_confirmations)
            print(result)
            
    def update_utxo(self, tx_id):
        with self._driver.session() as session:
            result = session.write_transaction(self._update_utxo, tx_id)
            print(result)
      

In [112]:
db = WalletDB( "neo4j://localhost:7687" , "neo4j" , "wallet" )


In [102]:
db.register_new_address("test_address1",0,False)
db.register_new_address("test_address2",777,False)
db.register_new_address("test_address3",22222,False)
db.register_new_address("test_address4",92747,True)
db.register_new_address("test_address5",99199191,True)
db.register_new_address("test_address6",625,True)

<Record a=<Path start=<Node id=13 labels={'address'} properties={'address': 'test_address1', 'type': 'recipient', 'acc_index': 0, 'created': 1586242324389}> end=<Node id=13 labels={'address'} properties={'address': 'test_address1', 'type': 'recipient', 'acc_index': 0, 'created': 1586242324389}> size=0>>
<Record a=<Path start=<Node id=14 labels={'address'} properties={'address': 'test_address2', 'type': 'recipient', 'acc_index': 777, 'created': 1586242324409}> end=<Node id=14 labels={'address'} properties={'address': 'test_address2', 'type': 'recipient', 'acc_index': 777, 'created': 1586242324409}> size=0>>
<Record a=<Path start=<Node id=15 labels={'address'} properties={'address': 'test_address3', 'type': 'recipient', 'acc_index': 22222, 'created': 1586242324416}> end=<Node id=15 labels={'address'} properties={'address': 'test_address3', 'type': 'recipient', 'acc_index': 22222, 'created': 1586242324416}> size=0>>
<Record a=<Path start=<Node id=7 labels={'address'} properties={'address'

In [103]:
db.register_new_utxo("test_address1",1000,2)
db.register_new_utxo("test_address2",1001,2)
db.register_new_utxo("test_address3",1002,2)
db.register_new_utxo("test_address4",1004,2)

<Record p=<Path start=<Node id=1 labels={'utxo'} properties={'transaction_id': 1000, 'out_index': 2, 'local_index': 0, 'address': 'test_address1', 'spent': False}> end=<Node id=13 labels={'address'} properties={'address': 'test_address1', 'type': 'recipient', 'created': 1586242324389, 'acc_index': 0}> size=1>>
<Record p=<Path start=<Node id=3 labels={'utxo'} properties={'transaction_id': 1001, 'out_index': 2, 'local_index': 1, 'address': 'test_address2', 'spent': False}> end=<Node id=14 labels={'address'} properties={'address': 'test_address2', 'type': 'recipient', 'created': 1586242324409, 'acc_index': 777}> size=1>>
<Record p=<Path start=<Node id=5 labels={'utxo'} properties={'transaction_id': 1002, 'out_index': 2, 'local_index': 2, 'address': 'test_address3', 'spent': False}> end=<Node id=15 labels={'address'} properties={'address': 'test_address3', 'type': 'recipient', 'created': 1586242324416, 'acc_index': 22222}> size=1>>
<Record p=<Path start=<Node id=6 labels={'utxo'} propertie

In [104]:
db.register_new_tx(2222,[0,1,2], ["ouptut1","output2"])

2222
[0, 1, 2]
['ouptut1', 'output2']
None


In [110]:
db.update_confirmations(2222,7)

<Record n=<Node id=9 labels={'TxOut'} properties={'outputs': ['ouptut1', 'output2'], 'id': 2222, 'confirmations': 7, 'created': 1586242324506, 'inputs': [0, 1, 2]}>>


In [113]:
db.update_utxo(2222)

<Record p=<Path start=<Node id=5 labels={'utxo'} properties={'transaction_id': 1002, 'out_index': 2, 'local_index': 2, 'address': 'test_address3', 'spent': True}> end=<Node id=9 labels={'TxOut'} properties={'outputs': ['ouptut1', 'output2'], 'id': 2222, 'confirmations': 7, 'created': 1586242324506, 'inputs': [0, 1, 2]}> size=1>>


/anaconda3/envs/mlenv/lib/python3.7/site-packages/neo4j/__init__.py:990: UserWarning: Expected a result with a single record, but this result contains 3
  warn("Expected a result with a single record, but this result contains %d" % size)
